In [45]:
import pandas as pd
import numpy as np

In [46]:
df = pd.read_csv('../data/interim/interm.csv')

In [47]:
df_cleaned = df

In [52]:
#Remove the extra column from dataset read from CSV
df_cleaned.drop(['Unnamed: 0'], axis=1, inplace=True)

In [48]:
from sklearn.preprocessing import OneHotEncoder

In [49]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [57]:
df_cleaned["Gender"] = pd.DataFrame(ohe.fit_transform(df_cleaned[["Gender"]]))

In [58]:
df_cleaned.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1.0,19,15,39
1,1.0,21,15,81
2,0.0,20,16,6
3,0.0,23,16,77
4,0.0,31,17,40


In [65]:
%load_ext autoreload
%autoreload 2

In [69]:
from src.data.sets import split_sets_random, save_sets

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(df=df_cleaned, target_col="Spending Score (1-100)")
save_sets(X_train, y_train, X_val, y_val, X_test, y_test)

In [1]:
import mlflow, mlflow.sklearn

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("xgboost_spending")

INFO: 'xgboost_spending' does not exist. Creating a new experiment


In [2]:
mlflow.start_run()

<ActiveRun: >

In [3]:
from src.data.sets import load_sets

X_train, y_train, X_val, y_val, X_test, y_test = load_sets()

In [4]:
mlflow.set_tag("model_description", "RandomForest with default hyperparameter")
mlflow.set_tag("model.version", "0.1")

In [5]:
mlflow.sklearn.autolog()

In [8]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=8)

In [9]:
rfr.fit(X_train, y_train)

2021/07/04 10:20:01 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'be95bb4bc39c43ba8df1ffcc74248c44'
2021/07/04 10:20:01 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'be95bb4bc39c43ba8df1ffcc74248c44'
2021/07/04 10:20:01 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run with ID 'be95bb4bc39c43ba8df1ffcc74248c44'
2021/07/04 10:20:01 INFO mlflow.utils.autologging_utils: sklearn autologging will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow to the MLflow run

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [10]:
from mlflow.models.signature import infer_signature

In [11]:
sig = infer_signature(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/mlflow/models/signature.py:124: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  outputs = _infer_schema(model_output) if model_output is not None else None


In [12]:
mlflow.sklearn.log_model (rfr, artifact_path="model", signature=sig, registered_model_name="sklearn-rf-spending")

Successfully registered model 'sklearn-rf-spending'.
2021/07/04 10:25:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sklearn-rf-spending, version 1
Created version '1' of model 'sklearn-rf-spending'.
